In [11]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 452 kB 52.6 MB/s 
     |████████████████████████████████| 7.6 MB 36.7 MB/s 
     |████████████████████████████████| 182 kB 55.1 MB/s 
     |████████████████████████████████| 132 kB 41.6 MB/s 
     |████████████████████████████████| 213 kB 58.5 MB/s 
     |████████████████████████████████| 127 kB 49.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [12]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0',data_dir="daily_mail")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0-data_dir=daily_mail/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
!cd drive/MyDrive

/bin/bash: line 0: cd: drive/MyDrive: No such file or directory


In [14]:
!ls

 big_law_data.txt		 'meeting有為火烤 (回覆).gsheet'
'Colab Notebooks'		  meeting菜同學.gform
 data_law_new.json		 'meeting菜同學 (回覆).gsheet'
 L4				 'meeting 貴族便當旗艦店 (回覆).gsheet'
 meetingTAKA炸雞.gform		 'meeting 貴族便當旗艦店(老闆很機車).gform'
'meetingTAKA炸雞 (回覆).gsheet'   meeting雲端私廚.gform
 meeting京鼎便當.gform		 'meeting雲端私廚 (回覆).gsheet'
'meeting京鼎便當 (回覆).gsheet'   model
'meeting台鐵便當 (回覆).gsheet'   nlplabbrief.pptx
 meeting坂東屋.gform		  NLPlab總務
 meeting小鋤堂.gform		  slide
'meeting小鋤堂 (回覆).gsheet'	  歷屆考古詳解-20211116T031833Z-001.zip
 meeting有為火烤.gform		  考古.zip


In [15]:
train_dataset=dataset["train"][0:5000]
val_dataset=dataset["validation"][0:500]
test_dataset=dataset["test"][0:500]

In [16]:
from transformers import  BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

tokenize_train_data=tokenizer(train_dataset['article'],text_target=train_dataset['highlights'],padding=True,truncation=True)
tokenize_val_data=tokenizer(val_dataset['article'],text_target=val_dataset['highlights'],padding=True,truncation=True)
tokenize_test_data=tokenizer(test_dataset['article'],text_target=test_dataset['highlights'],padding=True,truncation=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [17]:
tokenize_train_data.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [18]:
import torch
from torch.utils.data import DataLoader
class cnnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_data =cnnDataset(tokenize_train_data)
val_data =cnnDataset(tokenize_val_data)
test_data =cnnDataset(tokenize_test_data)

In [20]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 191 kB 5.2 MB/s 


In [21]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)


In [22]:
train_batch_size = 12      # 設定 training batch size
eval_batch_size = 12     # 設定 eval batch size
num_train_epochs = 12      # 設定 epoch

In [23]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_data, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(val_data, collate_fn=data_collator, batch_size=eval_batch_size)

In [24]:
print(len(train_dataloader))
print(len(train_data))

417
5000


In [25]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

learning_rate=1e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

max_train_steps 5004


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.1 MB/s 


In [30]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=3aa5db072b29c977f1f5b231896e9f24ce1fa1fce06aa88dbcb127df31baf7ce
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = 'model_bart/'


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "rouge": 0 }

import evaluate
rouge_score = evaluate.load("rouge")
for epoch in trange(num_train_epochs, desc="Epoch"):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1

        if step % 100 == 0:
            print({'epoch': epoch, 'step': step, 'loss': loss.item()})

        if completed_steps >= max_train_steps:
            break

    logger.info("***** Running eval *****")
    model.eval()
    preds=[]
    answers=[]
    for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
        outputs = model(**batch)
        output_ids = model.generate(batch['input_ids'], num_beams=2, max_length=150, early_stopping=True)
        preds.append(" ".join([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in output_ids]))
        answers.append(" ".join([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in batch['labels']]))
    
    
    scores = rouge_score.compute(
    predictions=preds, references=answers
    )
    print(scores)
    eval_precision=scores["rougeL"]
    logger.info(f"epoch :{epoch} \n rougeL:  {eval_precision}")
    if eval_precision > best_epoch["rouge"]:
        best_epoch['epoch'] = num_train_epochs
        best_epoch['rouge'] = eval_precision
    
if output_dir is not None :
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + 'final_model'+ '/', save_function=accelerator.save)

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Iteration:   0%|          | 0/417 [00:00<?, ?it/s]